# Parsing email
## Example data is from enron email dataset
- Public domain, widely used for demonstration purposes, considered canonical corpus of internal company emails.
- There are different formats of this data available in different places, but here we are looking at a single csv file with two columns.
- Each record only has a file location and raw text data, the latter of which contains metadata and the actual message content.
- Example code does some basic parsing of this raw text, separating the data from the metadata, using the Python `email` library, which is included in most basic Python distributions and in this runtime.
- 1000 records randomly seleted.
- Much more could be done, depending on the exact structure and form of a specific corpus of emails. Here we are just showing how this could be easily to then feed into SPSS Modeler.

In [1]:
import sys
import pandas as pd
import numpy as np
from project_lib import Project
project = Project.access()
import email 

In [2]:
raw_df = pd.read_csv('/project_data/data_asset/enron.csv')

In [3]:
raw_df.shape

(1000, 2)

In [4]:
raw_df.head()

file  \
0           shackleton-s/sent/6281.   
1      farmer-d/all_documents/2279.   
2  shackleton-s/all_documents/9796.   
3        germany-c/sent_items/1111.   
4       pereira-s/deleted_items/53.   

                                             message  
0  Message-ID: <28382053.1075844928453.JavaMail.e...  
1  Message-ID: <10878898.1075854020416.JavaMail.e...  
2  Message-ID: <9594692.1075844711064.JavaMail.ev...  
3  Message-ID: <930695.1075845149982.JavaMail.eva...  
4  Message-ID: <28380547.1075858910471.JavaMail.e...

## Here is what a raw email message looks like

In [5]:
print(raw_df.loc[0, ["message"]].values[0])

Message-ID: <28382053.1075844928453.JavaMail.evans@thyme>
Date: Tue, 29 May 2001 03:27:00 -0700 (PDT)
From: sara.shackleton@enron.com
To: lisa.mellencamp@enron.com
Subject: KCS Energy Marketing Inc. ("KCS")
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Sara Shackleton
X-To: Lisa Mellencamp
X-cc: 
X-bcc: 
X-Folder: \Sara_Shackleton_Dec2000_June2001_2\Notes Folders\Sent
X-Origin: SHACKLETON-S
X-FileName: sshackle.nsf

Lisa:

FYI, there has been activity regarding the KCS file, in particular, the 
status of the amendment to permit cross-netting of certain agreements and the 
impact of that amendment now that KCS has emerged from bankruptcy.  Two (2) 
new financial transactions have been executed and there is a VPP looming 
ahead for June 15.  The issue which we were trying to isolate is the effect 
of the amendment to the financial/physical agreements post-bankruptcy.  We 
concluded that KCS is bound post-bankruptcy (Carol St. Clair t

## Helper functions

In [6]:
def parse_emails(df, message_col):
    """
    High-level helper function to pass to df.pipe()
    Input: pandas dataframe with a column of raw email text, containt data and metadata
    Output: pandas dataframe with multiple columns of metadata and data (message body)
    
    """
    df = df.copy()
    msg_obj_series = df[message_col].apply(email.message_from_string)  # Using open-source python package in clean and easy way.
    df["subject"] = msg_obj_series.apply(lambda x: x["subject"])
    df["date"] = msg_obj_series.apply(lambda x: x["date"])
    df["recipient_raw"] = msg_obj_series.apply(lambda x: x["to"])
    df["x_to"] = msg_obj_series.apply(lambda x: x["X-To"])
    df["recipient"] = df["recipient_raw"].fillna(df["x_to"].apply(lambda x: str(x).lower()))
    df["sender"] = msg_obj_series.apply(lambda x: x["from"])
    df["clean_text"] = msg_obj_series.apply(extract_text)
    df["most_recent_msg"] = df["clean_text"].apply(lambda x: x.split("\n\n\n\n\n")[0])
    return df

def extract_text(msg):
    """
    Input: email.message object
    Output: raw text of email message, without metadata
    """
    if msg.is_multipart():
        for part in msg.walk():
            payload = part.get_payload(decode=True) #returns a bytes object
            text = payload.decode()
            return text
    else:
        payload = msg.get_payload(decode=True)
        text = payload.decode()
        return text

def is_internal(df, from_col, to_col):
    """
    Creates a Boolean flag representing whether all sender and recipents for an email are internal to enron
    Input: pandas dataframe with a sender colum and a recipient column
    Output: a pandas dataframe with an `is_internal` flag appended
    """
    df = df.copy()
    domain_from = df[from_col].apply(lambda x: x.split("@")[1] if x is not None else None)
    external_domain_flag = df[to_col].apply(lambda x: parse_to_field(x))
    df["is_internal"] = domain_from.str.startswith("enron") & external_domain_flag
    return df


def parse_to_field(contents):
    """
    Low-level helper for parsing multiple recipient email addresses; returns False if any is external
    Input: A string of multiple email addresses separated by a comma
    Output: A Boolean flag. True if all email address domains start with 'enron'
    
    """
    if contents is None:
        return contents
    try:
        domains = set([addr.split("@")[1] for addr in contents.split(",")])
    except:
#         print(contents)
        return np.nan
    for domain in domains:
        if domain.startswith("enron"):
            continue
        else:
            return False
    return True

### Testing `parse_to_field()` helper funtion
Should return `True` if all parties are internal (sender and recipients)

In [7]:
example_str_1 = 'joe@enron.com, jane@dow.com'
example_str_2 = 'joe@disney.com, jane@dow.com'
example_str_3 = 'joe@enron.com, jane@enron.com'

In [8]:
parse_to_field(example_str_1)

False

In [9]:
parse_to_field(example_str_2)

False

In [10]:
parse_to_field(example_str_3)

True

## Create preprocessed dataframe `prep_df`

In [11]:
prep_df = \
    (raw_df.copy()
        .pipe(parse_emails, "message")
        .assign(wc_full_text=lambda x: x["clean_text"].apply(lambda x: len(x.strip().split(' '))))
        .assign(wc_most_recent=lambda x: x["most_recent_msg"].apply(lambda x: len(x.strip().split(' '))))
        .pipe(is_internal, "sender", "recipient")
        .replace(r'^\s*$', np.nan, regex=True)
    )

In [12]:
prep_df.shape

(1000, 13)

In [13]:
prep_df.columns.tolist()

['file',
 'message',
 'subject',
 'date',
 'recipient_raw',
 'x_to',
 'recipient',
 'sender',
 'clean_text',
 'most_recent_msg',
 'wc_full_text',
 'wc_most_recent',
 'is_internal']

In [14]:
prep_df.head()

file  \
0           shackleton-s/sent/6281.   
1      farmer-d/all_documents/2279.   
2  shackleton-s/all_documents/9796.   
3        germany-c/sent_items/1111.   
4       pereira-s/deleted_items/53.   

                                             message  \
0  Message-ID: <28382053.1075844928453.JavaMail.e...   
1  Message-ID: <10878898.1075854020416.JavaMail.e...   
2  Message-ID: <9594692.1075844711064.JavaMail.ev...   
3  Message-ID: <930695.1075845149982.JavaMail.eva...   
4  Message-ID: <28380547.1075858910471.JavaMail.e...   

                                subject  \
0     KCS Energy Marketing Inc. ("KCS")   
1                                 Help!   
2  (01-92) Amendments to Delivery Rules   
3                             RE: Taxes   
4     New Link for All-Employee Meeting   

                                    date  \
0  Tue, 29 May 2001 03:27:00 -0700 (PDT)   
1  Tue, 21 Mar 2000 05:32:00 -0800 (PST)   
2  Wed, 14 Mar 2001 06:52:00 -0800 (PST)   
3  Fri, 20 Apr 2001 15:50:00 -0700 (PDT)   
4  Mon, 22 Oct 2001 21:00:26 -0700 (PDT)   

                               recipient_raw  \
0                  lisa.mellencamp@enron.com   
1                   brian.heinrich@enron.com   
2                  sara.shackleton@enron.com   
3  germany.jr@enron.com, wdgermanyjr@dow.com   
4                                       None   

                                          x_to  \
0                              Lisa Mellencamp   
1                               Brian Heinrich   
2                    sara.shackleton@enron.com   
3  Germany Jr, Don (WD)  <WDGERMANYJR@dow.com>   
4                    All Enron Worldwide@ENRON   

                                   recipient                     sender  \
0                  lisa.mellencamp@enron.com  sara.shackleton@enron.com   
1                   brian.heinrich@enron.com  yvette.connevey@enron.com   
2                  sara.shackleton@enron.com     exchangeinfo@nymex.com   
3  germany.jr@enron.com, wdgermanyjr@dow.com    chris.germany@enron.com   
4                  all enron worldwide@enron       no.address@enron.com   

                                          clean_text  \
0  Lisa:\n\nFYI, there has been activity regardin...   
1  Brian, \n\nWho do I talk to about getting Dare...   
2  Notice No. 01-92\nMarch 14, 2001\n\n\nTO:\nAll...   
3  This is what I want, at 17 to 18 footer with a...   
4  Attached is a new link for employees unable to...   

                                     most_recent_msg  wc_full_text  \
0  Lisa:\n\nFYI, there has been activity regardin...           113   
1  Brian, \n\nWho do I talk to about getting Dare...            51   
2  Notice No. 01-92\nMarch 14, 2001\n\n\nTO:\nAll...           982   
3  This is what I want, at 17 to 18 footer with a...           409   
4  Attached is a new link for employees unable to...            48   

   wc_most_recent  is_internal  
0             113         True  
1              51         True  
2             154        False  
3              39        False  
4              48         True

### Flag for internal emails seems to work

In [15]:
prep_df.filter(["recipient", "sender", "is_internal"]).head(10)

recipient                      sender  \
0                  lisa.mellencamp@enron.com   sara.shackleton@enron.com   
1                   brian.heinrich@enron.com   yvette.connevey@enron.com   
2                  sara.shackleton@enron.com      exchangeinfo@nymex.com   
3  germany.jr@enron.com, wdgermanyjr@dow.com     chris.germany@enron.com   
4                  all enron worldwide@enron        no.address@enron.com   
5                   gary.hickerson@enron.com  jeffrey.shankman@enron.com   
6                     daren.farmer@enron.com       lisa.csikos@enron.com   
7                         j.harris@enron.com     james.derrick@enron.com   
8              darrell.schoolcraft@enron.com    enerfaxdaily@enerfax.com   
9                        jsmith@austintx.com     phillip.allen@enron.com   

   is_internal  
0         True  
1         True  
2        False  
3        False  
4         True  
5         True  
6         True  
7         True  
8        False  
9        False

### We are filling in missing `To` field with `X-To` field.
Still missing the email recipients for 20 rows. Could do additional EDA and parsing

In [16]:
prep_df.isnull().sum()

file                0
message             0
subject            39
date                0
recipient_raw      53
x_to               20
recipient          20
sender              0
clean_text          0
most_recent_msg     0
wc_full_text        0
wc_most_recent      0
is_internal         0
dtype: int64

In [17]:
no_recipients_df = prep_df.copy().query("recipient_raw != recipient_raw")

In [18]:
no_recipients_df[["recipient_raw", "x_to", "recipient", "is_internal"]].head(10)

recipient_raw                                               x_to  \
4            None                          All Enron Worldwide@ENRON   
21           None  rodc@canion.com, jharris@echoventures.com, drt...   
35           None      All Enron Employees United States Group@ENRON   
41           None                            All Enron Houston@ENRON   
51           None  All Enron Worldwide@ENRON <??SAll Enron Worldw...   
60           None  All Enron Worldwide@ENRON <??SAll Enron Worldw...   
146          None                                                NaN   
197          None                                                NaN   
199          None                                                NaN   
206          None                          All Enron Worldwide@ENRON   

                                             recipient  is_internal  
4                            all enron worldwide@enron         True  
21   rodc@canion.com, jharris@echoventures.com, drt...        False  
35       all enron employees united states group@enron         True  
41                             all enron houston@enron         True  
51   all enron worldwide@enron <??sall enron worldw...         True  
60   all enron worldwide@enron <??sall enron worldw...         True  
146                                                NaN        False  
197                                                NaN        False  
199                                                NaN        False  
206                          all enron worldwide@enron         True

In [19]:
no_recipients_df[["recipient_raw", "x_to", "recipient", "is_internal"]].query("recipient != recipient")

recipient_raw x_to recipient  is_internal
146          None  NaN       NaN        False
197          None  NaN       NaN        False
199          None  NaN       NaN        False
221          None  NaN       NaN        False
242          None  NaN       NaN        False
348          None  NaN       NaN        False
351          None  NaN       NaN        False
438          None  NaN       NaN        False
455          None  NaN       NaN        False
483          None  NaN       NaN        False
496          None  NaN       NaN        False
500          None  NaN       NaN        False
513          None  NaN       NaN        False
664          None  NaN       NaN        False
674          None  NaN       NaN        False
704          None  NaN       NaN        False
785          None  NaN       NaN        False
816          None  NaN       NaN        False
822          None  NaN       NaN        False
987          None  NaN       NaN        False

In [20]:
no_recipients_df.loc[51, ["x_to"]].values[0]

'All Enron Worldwide@ENRON <??SAll Enron Worldwide@ENRON>'

## Comparing raw data to preprocessed data

In [21]:
example_idx = 3

In [22]:
example_message = raw_df.loc[example_idx, "message"]

### Raw text of example message

In [23]:
example_message

'Message-ID: <930695.1075845149982.JavaMail.evans@thyme>\nDate: Fri, 20 Apr 2001 15:50:00 -0700 (PDT)\nFrom: chris.germany@enron.com\nTo: germany.jr@enron.com, wdgermanyjr@dow.com\nSubject: RE: Taxes\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Chris Germany\nX-To: Germany Jr, Don (WD)  <WDGERMANYJR@dow.com>\nX-cc: \nX-bcc: \nX-Folder: \\Germany, Chris\\Germany, Chris\\Sent Items\nX-Origin: GERMANY-C\nX-FileName: Germany, Chris.pst\n\nThis is what I want, at 17 to 18 footer with a 70-100 HP motor.  I prefer a shallow draft center console for bay or bayou fishing.\n\nMake it happen buddy - and remember, I\'m not made of money.\n\n\n\n\n"Germany Jr, Don (WD)" <WDGERMANYJR@dow.com> on 04/16/2001 07:03:27 AM\nTo:\t"\'Chris.Germany@enron.com\'" <Chris.Germany@enron.com>\ncc:\t \nSubject:\tRE: Taxes\n\n\nI am glad to hear the both of you are o.k.  How does the grass look up at\nDad\'s.  I am thinking we should maybe bale the hay.  I

### Pretty-Printing Example Raw Message

In [24]:
print(example_message)

Message-ID: <930695.1075845149982.JavaMail.evans@thyme>
Date: Fri, 20 Apr 2001 15:50:00 -0700 (PDT)
From: chris.germany@enron.com
To: germany.jr@enron.com, wdgermanyjr@dow.com
Subject: RE: Taxes
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Chris Germany
X-To: Germany Jr, Don (WD)  <WDGERMANYJR@dow.com>
X-cc: 
X-bcc: 
X-Folder: \Germany, Chris\Germany, Chris\Sent Items
X-Origin: GERMANY-C
X-FileName: Germany, Chris.pst

This is what I want, at 17 to 18 footer with a 70-100 HP motor.  I prefer a shallow draft center console for bay or bayou fishing.

Make it happen buddy - and remember, I'm not made of money.




"Germany Jr, Don (WD)" <WDGERMANYJR@dow.com> on 04/16/2001 07:03:27 AM
To:	"'Chris.Germany@enron.com'" <Chris.Germany@enron.com>
cc:	 
Subject:	RE: Taxes


I am glad to hear the both of you are o.k.  How does the grass look up at
Dad's.  I am thinking we should maybe bale the hay.  I have intentions of
going up there this c

### Cleaned message text (top-level metadata extracted)
- Messages contain entire email thread, from original message to the most recent message
- This could result in duplication, so making an attempt to keep only the most recent message in the thread.

In [25]:
print(prep_df.loc[example_idx, ["clean_text"]].values[0])

This is what I want, at 17 to 18 footer with a 70-100 HP motor.  I prefer a shallow draft center console for bay or bayou fishing.

Make it happen buddy - and remember, I'm not made of money.




"Germany Jr, Don (WD)" <WDGERMANYJR@dow.com> on 04/16/2001 07:03:27 AM
To:	"'Chris.Germany@enron.com'" <Chris.Germany@enron.com>
cc:	 
Subject:	RE: Taxes


I am glad to hear the both of you are o.k.  How does the grass look up at
Dad's.  I am thinking we should maybe bale the hay.  I have intentions of
going up there this coming weekend.  You should have bought a boat during
the boat show.  I have a friend who is always fishing and I will ask him
about a boat.  Do you want a ski boat or a fishing boat?  Freshwater or
Saltwater.  I do know the best motors are made by Yamaha.  A friend of mine
at Demi Jon had a Gulf Coast which is basically a chopped down Boston
Whaler.  I think the boat was around 21/23 '.  The sides were approximately
6/8" tall with a center console and a very large ice chest 

### Cleaned message text, most recent message only:

In [26]:
print(prep_df.loc[example_idx,  ["most_recent_msg"]].values[0])

This is what I want, at 17 to 18 footer with a 70-100 HP motor.  I prefer a shallow draft center console for bay or bayou fishing.

Make it happen buddy - and remember, I'm not made of money.


In [27]:
prep_df.loc[example_idx]

file                                      germany-c/sent_items/1111.
message            Message-ID: <930695.1075845149982.JavaMail.eva...
subject                                                    RE: Taxes
date                           Fri, 20 Apr 2001 15:50:00 -0700 (PDT)
recipient_raw              germany.jr@enron.com, wdgermanyjr@dow.com
x_to                     Germany Jr, Don (WD)  <WDGERMANYJR@dow.com>
recipient                  germany.jr@enron.com, wdgermanyjr@dow.com
sender                                       chris.germany@enron.com
clean_text         This is what I want, at 17 to 18 footer with a...
most_recent_msg    This is what I want, at 17 to 18 footer with a...
wc_full_text                                                     409
wc_most_recent                                                    39
is_internal                                                    False
Name: 3, dtype: object

### There are some edge-cases for the straightforward approach we used to grab the first message

#### For instance, most of the longest messages are actually compilations of short news articles

#### ...  And the straightforward approach for capturing the most recent message also only captures the first news article in the list

#### Each dataset has its own issues. Something like this could be worked through, and decisions made based on the questions you wanted to answer

(E.g., Should these emails be included at all?)

If we're dealing purely with customer emails, this issue wouldn't exist the way it does in the Enron email dataset.)

In [28]:
prep_df.assign(wc_diff=lambda x: x["wc_full_text"] - x["wc_most_recent"]).filter(["wc_full_text", "wc_most_recent","is_internal", "wc_diff", "subject"]).sort_values("wc_diff", ascending=False).head(10)

wc_full_text  wc_most_recent  is_internal  wc_diff  \
455         14648             307        False    14341   
151         12875             803        False    12072   
438         10084             165        False     9919   
980          1532             281        False     1251   
956          1271             122         True     1149   
851          1103             127         True      976   
331          1195             221        False      974   
237          1127             208         True      919   
65            915              51         True      864   
112           880              23        False      857   

                                               subject  
455                                     Enron Mentions  
151                                      Energy Issues  
438                                     Enron Mentions  
980  Today's Headlines from NYTimes.com Tuesday, Ja...  
956                            Re: FERC Data Warehouse  
851  Re: ANSWERING YOUR QUESTIONS ON CA TRANSMISSIO...  
331                     FW: True Orange E-Mail-Fax #61  
237  Re: Citygate Production GAS behind TCO - PLEAS...  
65   RE: FW: Transmission curtailments - PG&E Proposal  
112                  California Power Issue Article II

### Looking at the first five thousand characters of a very long email (a list of news articles)

In [29]:
prep_df.iloc[151, :]

file                                 dasovich-j/all_documents/28811.
message            Message-ID: <32251663.1075849323688.JavaMail.e...
subject                                                Energy Issues
date                           Tue, 17 Jul 2001 02:56:00 -0700 (PDT)
recipient_raw      joseph.alamo@enron.com, bhansen@lhom.com, rob....
x_to               Joseph Alamo, bhansen <bhansen@lhom.com>@SMTP@...
recipient          joseph.alamo@enron.com, bhansen@lhom.com, rob....
sender                                       miyung.buster@enron.com
clean_text         Please see the following articles:\n\nSac Bee,...
most_recent_msg    Please see the following articles:\n\nSac Bee,...
wc_full_text                                                   12875
wc_most_recent                                                   803
is_internal                                                    False
Name: 151, dtype: object

In [30]:
print(prep_df.loc[151, ["message"]].values[0][:5000])

Message-ID: <32251663.1075849323688.JavaMail.evans@thyme>
Date: Tue, 17 Jul 2001 02:56:00 -0700 (PDT)
From: miyung.buster@enron.com
To: joseph.alamo@enron.com, bhansen@lhom.com, rob.bradley@enron.com, 
	tom.briggs@enron.com, michael.brown@enron.com, 
	janet.butler@enron.com, stella.chan@enron.com, alan.comnes@enron.com, 
	shelley.corman@enron.com, jeff.dasovich@enron.com, 
	larry.decker@enron.com, karen.denne@enron.com, 
	marcus.dotson@enron.com, filuntz@aol.com, markus.fiala@enron.com, 
	lamar.frazier@enron.com, janel.guerrero@enron.com, 
	joe.hartsoe@enron.com, hgovenar@govadv.com, chris.holmes@enron.com, 
	shirley.hudler@enron.com, evan.hughes@enron.com, 
	jkradin@marathon-com.com, steven.kean@enron.com, 
	ken@kdscommunications.com, amy.kim@enron.com, kurt.lindahl@enron.com, 
	elizabeth.linnell@enron.com, lipsen@cisco.com, chris.long@enron.com, 
	peggy.mahoney@enron.com, jeannie.mandelker@enron.com, 
	susan.mara@enron.com, luiz.maurer@enron.com, karen.moore@enron.com, 
	andrew.morri

In [31]:
print(prep_df.loc[151, ["most_recent_msg"]].values[0][:5000])

Please see the following articles:

Sac Bee, Tues, 7/17: Jones demands probe of state's energy buyers 
Sac Bee, Tues, 7/17: Energy Digest: PG&E's managers will get bonuses
SD Union, Tues, 7/17: Bush's top team launches blitz for his energy agenda 
SD Union, Tues, 7/17: Davis aide foresees an end to rate hikes 
SD Union, Tues, 7/17: Bankruptcy judge lets PG&E pay managers $17.5 million 
bonus 
SD Union, Tues, 7/17: As power crisis eases, Davis sees a beneficial role for 
deregulation 
LA Times, Tues, 7/17: Churches Help Edison Aid the Poor
SF Chron, Tues, 7/17: Consumers await word on rates 
Report's delay, government bond issue raise questions about electricity price 
increase 
Mercury News, Tues, 7/17: Judge OKs bonuses for PG&E chiefs 
OC Register, Tues, 7/17: Power of lies    (Commentary) 
LA Times, Tues, 7/17: THE NATION State Losing Ground in War on Dirty Air 
Environment:
Growth, lax enforcement are blamed for rising smog levels in some areas
LA Times, Tues, 7/17: THE NATION In S

## Saving preprocessed dataframe

In [32]:
project.save_data(file_name="preprocessed.csv", data=prep_df.to_csv(), overwrite=True)

{'file_name': 'preprocessed.csv',
 'message': 'File saved to project storage.',
 'asset_id': '78ac0090-0e8b-4f79-86ca-f91992e2ed2e'}